In [2]:
!pip install pandas

In [9]:
import pandas as pd

def mostrar_dados_github(url_github):
  """
  Abre um arquivo CSV diretamente de um URL do GitHub e exibe todas as linhas
  em formato de tabela, sem quebras.

  Args:
    url_github (str): O URL raw do arquivo CSV no GitHub.
  """
  try:
    df = pd.read_csv(url_github)
    # Configura o pandas para mostrar todas as linhas e colunas sem truncamento
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    print(df.to_string(index=False))  # Exibe o DataFrame como string sem o índice
  except Exception as e:
    print(f"Ocorreu um erro ao acessar ou ler o arquivo: {e}")

# Exemplo de uso com o URL fornecido
url = "https://raw.githubusercontent.com/josecacciatore89/acidentes_aereos/refs/heads/main/dados_limpos.csv"
mostrar_dados_github(url)

 Ano        Data do acidente               País do acidente                                              Modelo da aeronave                                           Número de mortos
1919     2 de agosto de 1919                         Itália                                                  Caproni Ca. 48                                                       13.5
1920          14 de dezembro                     Inglaterra                                         Handley Page Type O-400                                                        4.0
1922      7 de abril de 1922                         França                                                Havilland DH.18A                                                        7.0
1923      14 de maio de 1923                         França                                             Farman F.60 Goliath                                                        6.0
1923    27 de agosto de 1923                     Inglaterra                          

In [15]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [18]:
import pandas as pd
from google.colab import files

def gerar_tabela_interativa_html(url_github, nome_arquivo_html='tabela_acidentes_interativa.html'):
    """
    Abre um arquivo CSV do GitHub e gera um arquivo HTML contendo uma tabela
    interativa com filtragem por país usando JavaScript, convertendo todos os
    dados para string para evitar erros de comparação de tipos.

    Args:
        url_github (str): O URL raw do arquivo CSV no GitHub.
        nome_arquivo_html (str, opcional): O nome do arquivo HTML a ser salvo.
                                           Padrão é 'tabela_acidentes_interativa.html'.
    """
    try:
        df = pd.read_csv(url_github)
        df_selecionado = df[['Ano', 'Data do acidente', 'País do acidente', 'Modelo da aeronave', 'Número de mortos']].copy()
        df_selecionado.rename(columns={'Data do acidente': 'Data_Acidente',
                                       'País do acidente': 'Pais_Acidente',
                                       'Modelo da aeronave': 'Modelo_Aeronave',
                                       'Número de mortos': 'Numero_Mortos'}, inplace=True)

        # Converter todos os valores do DataFrame para string
        df_selecionado = df_selecionado.astype(str)

        # Estilos CSS inline para a tabela
        table_style = {
            'width': '90%',
            'margin': '20px auto',
            'borderCollapse': 'collapse'
        }
        header_style = {
            'backgroundColor': 'lightgrey',
            'fontWeight': 'bold',
            'padding': '8px',
            'border': '1px solid #ddd',
            'textAlign': 'left'
        }
        cell_style = {
            'padding': '8px',
            'border': '1px solid #ddd',
            'textAlign': 'left'
        }
        dropdown_style = {
            'width': '50%',
            'margin': '10px auto'
        }
        container_style = {
            'fontFamily': 'Arial, sans-serif'
        }
        title_style = {
            'textAlign': 'center',
            'color': '#333'
        }

        # Criar a estrutura HTML da tabela e do dropdown diretamente
        dropdown_options_html = "".join(
            f'<option value="{pais}">{pais}</option>'
            for pais in sorted(df_selecionado['Pais_Acidente'].unique())
        )

        table_header_html = "<tr>" + "".join(f'<th style="{header_style}">{col}</th>' for col in df_selecionado.columns) + "</tr>"
        table_rows_html = "".join(
            "<tr>" + "".join(f'<td style="{cell_style}">{row[col]}</td>' for col in df_selecionado.columns) + "</tr>"
            for _, row in df_selecionado.iterrows()
        )

        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Tabela Interativa de Acidentes Aéreos</title>
            <style>
                body {{ font-family: Arial, sans-serif; }}
                h4 {{ text-align: center; color: #333; }}
                .container {{ width: 90%; margin: 20px auto; }}
                #filtro-pais {{ width: 50%; margin: 10px auto; display: block; padding: 8px; border: 1px solid #ccc; }}
                #tabela-acidentes {{ width: 100%; border-collapse: collapse; }}
                #tabela-acidentes th {{ background-color: lightgrey; font-weight: bold; padding: 8px; border: 1px solid #ddd; text-align: left; }}
                #tabela-acidentes td {{ padding: 8px; border: 1px solid #ddd; text-align: left; }}
                .hidden-row {{ display: none; }}
            </style>
        </head>
        <body>
            <h4>Dados de Acidentes Aéreos com Filtragem por País</h4>
            <select id="filtro-pais" onchange="filtrarTabela()">
                <option value="">Todos os Países</option>
                {dropdown_options_html}
            </select>
            <table id="tabela-acidentes">
                <thead>
                    {table_header_html}
                </thead>
                <tbody>
                    {table_rows_html}
                </tbody>
            </table>

            <script>
                function filtrarTabela() {{
                    var selectBox = document.getElementById("filtro-pais");
                    var selectedPais = selectBox.value;
                    var table = document.getElementById("tabela-acidentes").getElementsByTagName('tbody')[0];
                    var rows = table.getElementsByTagName('tr');

                    for (var i = 0; i < rows.length; i++) {{
                        var paisColuna = rows[i].getElementsByTagName('td')[2]; // Índice da coluna 'Pais_Acidente'

                        if (paisColuna) {{
                            var paisTexto = paisColuna.textContent || paisColuna.innerText;
                            if (selectedPais === "" || paisTexto === selectedPais) {{
                                rows[i].classList.remove("hidden-row");
                            }} else {{
                                rows[i].classList.add("hidden-row");
                            }}
                        }}
                    }}
                }}
            </script>
        </body>
        </html>
        """

        # Salvar o conteúdo HTML em um arquivo
        with open(nome_arquivo_html, "w") as f:
            f.write(html_content)

        print(f"Tabela interativa salva em: {nome_arquivo_html}")

        # Para download no Colab
        files.download(nome_arquivo_html)

    except Exception as e:
        print(f"Ocorreu um erro ao processar os dados ou gerar o HTML: {e}")

# Exemplo de uso com o URL fornecido
url = "https://raw.githubusercontent.com/josecacciatore89/acidentes_aereos/refs/heads/main/dados_limpos.csv"
gerar_tabela_interativa_html(url)

Tabela interativa salva em: tabela_acidentes_interativa.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>